In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [94]:
# code adapted from ChatGPT

# Define your neural network model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(2, 1, bias=False)  # Input size 2, output size 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        # RUN IT ON A GPU if it exists
        if torch.cuda.is_available():
            x = x.to("cuda")

        x = self.fc(x)
        x = self.sigmoid(x)
        
        return x

def train_model(inputs, desired_outputs, num_epochs=100, learning_rate=0.01, momentum=0.9):
    # Convert inputs and desired_outputs to PyTorch tensors
    inputs = torch.tensor(inputs, dtype=torch.float32)
    desired_outputs = torch.tensor(desired_outputs, dtype=torch.float32)
    
    # Create a DataLoader to handle batching (if needed)
    dataset = TensorDataset(inputs, desired_outputs)
    dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)  # Adjust batch_size as needed
    
    # Initialize the model
    model = SimpleModel()
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()  # Mean Squared Error loss
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    
    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_inputs, batch_desired_outputs in dataloader:
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(batch_inputs)  # Forward pass
            loss = criterion(outputs, batch_desired_outputs)  # Compute the loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update the model's parameters
            cur_item = loss.item()
            total_loss += cur_item
            # print(cur_item)
        
        # Print the average loss for this epoch
        print(f"total loss: {total_loss}")
        average_loss = total_loss / len(dataloader)
        for i, param in model.named_parameters():
            print(param)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}')
    
    # Return the trained model
    return model

In [96]:
input_data = np.random.randint(1, 100, size=(1000, 2))
desired_output = []

for i in input_data:
    desired_output.append(int(i[0] >= i[1]))
    
# Train the model
trained_model = train_model(input_data, desired_output, 200)
    


total loss: 0.5047408938407898
Parameter containing:
tensor([[-0.0107, -0.5560]], requires_grad=True)
Epoch [1/200], Loss: 0.5047
total loss: 0.5044677257537842
Parameter containing:
tensor([[-0.0075, -0.5558]], requires_grad=True)
Epoch [2/200], Loss: 0.5045
total loss: 0.5038995146751404
Parameter containing:
tensor([[-0.0028, -0.5554]], requires_grad=True)
Epoch [3/200], Loss: 0.5039
total loss: 0.502982497215271
Parameter containing:
tensor([[ 0.0034, -0.5550]], requires_grad=True)
Epoch [4/200], Loss: 0.5030
total loss: 0.5016816854476929
Parameter containing:
tensor([[ 0.0111, -0.5544]], requires_grad=True)
Epoch [5/200], Loss: 0.5017
total loss: 0.5000602602958679
Parameter containing:
tensor([[ 0.0201, -0.5537]], requires_grad=True)
Epoch [6/200], Loss: 0.5001
total loss: 0.4982619285583496
Parameter containing:
tensor([[ 0.0301, -0.5528]], requires_grad=True)
Epoch [7/200], Loss: 0.4983
total loss: 0.4964558482170105
Parameter containing:
tensor([[ 0.0408, -0.5517]], requires_

In [97]:
# Test the trained model (you can use new input data)
test_input = torch.tensor([[10, 20], [2, 1], [3, 5]], dtype=torch.float32)
predictions = trained_model(test_input)
print(predictions)

tensor([[0.3423],
        [0.5047],
        [0.4635]], grad_fn=<SigmoidBackward0>)


In [46]:
trained_model.named_parameters()

<generator object Module.named_parameters at 0x0000021A30BA63B0>

In [47]:
for i, param in trained_model.named_parameters():
    print(i)
    print(param)

fc.weight
Parameter containing:
tensor([[0.4275, 0.3911]], requires_grad=True)


In [ ]:
def count_model_params():
    '''
    return the number of trainable parameters of LeNet.
    '''
    model = SimpleModel()
    model_params = 0.0
    for _, parameter in model.named_parameters():
        if (parameter.requires_grad):
            model_params += parameter.numel()
    return model_params